In [20]:
import pandas as pd

In [21]:
# Open up excel into an object
# This takes a long time - in production might use different approach
xls = pd.ExcelFile('tyb2122.xlsx')

# Read sheets into a dictionary of sheets
excel_map = pd.read_excel(xls, sheet_name=None)
#print(excel_map.keys())

In [22]:
# Copy into a dict just because of naming below. No real need to do this
df_dict = excel_map

In [23]:
#print(df_dict['May 21'])

In [24]:
# Create a list of sheets to ignore
# Iterate over the sheets in the dict created above (misnamed stuff when starting out is all)
# Exclude the sheet if it's in the list 
# Otherwise - Group the data in the current sheet. Convert in-line to a dataframe
# Concatenate these as we go along
# Group the final sheet up to category level as it is the complete set
# Write out to csv
exclude_sheets = ['Template', 'Lists']

month_totals_df= pd.DataFrame()
totals_df= pd.DataFrame()

for k in df_dict.keys():
    #print(k)
    if k in exclude_sheets:
        print("Excluded")
        continue
    else:
        try:
            current_month_df = pd.DataFrame(df_dict[k].groupby('Category')['Actual'].sum())
        except:
            print(k)
        #Add current month to new dataframe
        current_month_df['month'] = k
        #display(current_month_df.head(10))
        #display(totals_df)
        month_totals_df = pd.concat([month_totals_df, current_month_df], axis=0, ignore_index=False)

# Group the months
totals_df = pd.DataFrame(month_totals_df.groupby('Category')['Actual'].sum()).reset_index()
print(totals_df)
rental_categories = ['Airbnb', 'Cottages', 'Private']
# output to csv
month_totals_df.to_csv('categories_all_months.csv')

totals_df.to_csv('category_totals.csv')


Excluded
Excluded
       Category    Actual
0   Advertising    -60.00
1        Airbnb  14888.25
2     Broadband   -327.98
3      Cottages   3675.52
4   Council Tax      0.00
5   Electricity  -2066.96
6   Furnishings  -1713.00
7     Household   -519.29
8     Insurance   -287.79
9       Laundry  -1492.70
10         Loan  -5317.34
11         Logs   -170.00
12  Maintenance   -918.62
13         Misc    -14.29
14     Mortgage  -2892.84
15      Private   3479.50
16     Private       0.00
17   TV Licence   -158.94
18        Water   -300.00
19      Welcome   -894.22


In [25]:
print(totals_df.head(10))


      Category    Actual
0  Advertising    -60.00
1       Airbnb  14888.25
2    Broadband   -327.98
3     Cottages   3675.52
4  Council Tax      0.00
5  Electricity  -2066.96
6  Furnishings  -1713.00
7    Household   -519.29
8    Insurance   -287.79
9      Laundry  -1492.70


april_df = df['April 21']
print(april_df.head(5))
april_df.columns = ['Date', 'Category', 'Detail', 'In / Out', 'Amount', 'Actual', 'Notes',
       'Unnamed: 7', 'Unnamed: 8']
print(april_df.head(5))

In [27]:
march_df = df_dict['March 22']

In [29]:
month_df = df_dict['May 21']
display(month_df)
per_month_actuals = month_df.groupby('Category')['Actual'].sum()
per_month_actuals.head(100)



,Date,Category,Detail,In / Out,Amount,Actual,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,2021-05-04 00:00:00,Mortgage,Principality,-,241.07,-241.07,NaN,NaN,NaN
1,2021-05-01 00:00:00,Council Tax,Gywnedd,-,0.00,0.00,NaN,NaN,NaN
2,2021-05-26 00:00:00,Electricity,Octopus,-,61.87,-61.87,NaN,NaN,NaN
3,2021-05-10 00:00:00,Water,Dwr Cymru,-,25.00,-25.00,NaN,In,2558.09
4,2021-05-14 00:00:00,Broadband,Vodafone,-,27.68,-27.68,NaN,Out,-1057.56
5,2021-05-06 00:00:00,Loan,M&S,-,179.06,-179.06,NaN,NaN,NaN
6,2021-05-20 00:00:00,Furnishings,DFS Sofa,-,25.66,-25.66,NaN,Total,1500.53
7,2021-05-04 00:00:00,TV Licence,TV,-,11.87,-11.87,NaN,NaN,NaN
8,2021-05-02 00:00:00,Laundry,Pressing Matters,-,131.50,-131.50,NaN,NaN,NaN
9,2021-05-30 00:00:00,Laundry,Pressing Matters,-,109.70,-109.70,NaN,NaN,NaN


Category
Airbnb         1229.10
Broadband       -27.68
Cottages        905.24
Council Tax       0.00
Electricity     -61.87
Furnishings    -138.07
Household       -24.20
Laundry        -241.20
Loan           -179.06
Maintenance     -62.41
Mortgage       -241.07
Private         423.75
TV Licence      -11.87
Water           -25.00
Welcome         -45.13
Name: Actual, dtype: float64

In [ ]:
# TODO Per month
